In [ ]:
 import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

#Read the dataset


In [ ]:
import pandas as pd
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

In [ ]:
liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)

values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'].replace(values_to_remove, replacement_value, inplace=True)

liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False)
liu_['Comments'] = liu_['Comments'].str.replace('"', '', regex=False)

In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")


Number of duplicate rows: 0


##CountVect

In [ ]:


data = liu_

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data['Comments'], data['TDType'], test_size=0.2, random_state=42)
# Further split the training set into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('CountVectorizer', CountVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),  # Scale feature vectors
    ('clf', SVC(random_state=42, kernel='linear', class_weight='balanced', probability=True))  # SVM
])


param_grid = {
    'clf__C': [0.01, 1],  # Regularization strength


}

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

# Fit the grid search to the training data
grid_search.fit(X_train_final, y_train_final)


print("Best parameters found:", grid_search.best_params_)

# Evaluate the model on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
classification_rep_val = classification_report(y_val, y_val_pred)

print("\nValidation Confusion Matrix:")
print(conf_matrix_val)
print("\nValidation Classification Report:")
print(classification_rep_val)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classification_rep_test = classification_report(y_test, y_test_pred)

print("\nTest Confusion Matrix:")
print(conf_matrix_test)
print("\nTest Classification Report:")
print(classification_rep_test)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters found: {'clf__C': 0.01}

Validation Confusion Matrix:
[[  53    3    6   54    1    7    0   19]
 [   1   36    3   15    0    5    0    7]
 [   1    4   54   19    4    5    5    7]
 [  74   32   68 1282   15  108   22  115]
 [   1    1    0    3   11    3    1    0]
 [   8    5    6   82    3  228    4   12]
 [   1    2    3   13    2    3   74    7]
 [  42   19    8   85    4    8    7 3549]]

Validation Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.29      0.37      0.33       143
         COMPATIBILITY       0.35      0.54      0.43        67
                DEFECT       0.36      0.55      0.44        99
                DESIGN       0.83      0.75      0.78      1716
         DOCUMENTATION       0.28      0.55      0.37        20
        IMPLEMENTATION       0.62      0.66      0.64       348
                  TEST       0.65      0.